# Integrate app into UI interface: Gradio

- **input fields**: 
    - text box for user to describe current game information
        - optional: voice-to-text capability
    - optional: dedicated input fields for current game information (pitcher name, batter name, who is on base, fielding alignment, etc)
- **output fields**:
    - LLM Chat with history
    - optional: toggle on/off history visibility

In [35]:
-m pip install --upgrade pip

SyntaxError: invalid syntax (2350214200.py, line 1)

In [36]:
!pip install --upgrade transformers datasets[audio] accelerate


     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     --------------------------------- ---- 51.2/57.6 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 607.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/330.9 kB ? eta -:--:--
   --- ----------------------------------- 30.7/330.9 kB 660.6 kB/s eta 0:00:01
   ---- ---------------------------------- 41.0/330.9 kB 495.5 kB/s eta 0:00:01
   -------- ------------------------------ 71.7/330.9 kB 491.5 kB/s eta 0:00:01
   ---------- ---------------------------- 92.2/330.9 kB 525.1 kB/s eta 0:00:01
   ------------ ------------------------- 112.6/330.9 kB 547.6 kB/s eta 0:00:01
   ---------------- --------------------- 143.4/330.9 kB 502.3 kB/s eta 0:00:01
   ----------------- -------------------- 153.6/330.9 kB 510.2 kB/s eta 0:00:01
   ---------------------- --------------- 194.6/330.9 kB 535.8 k

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\elcoo\\anaconda3\\lib\\site-packages\\__pycache__\\isympy.cpython-311.pyc'



  Using cached accelerate-1.0.1-py3-none-any.whl.metadata (19 kB)
  Using cached datasets-3.0.2-py3-none-any.whl.metadata (20 kB)
  Using cached pyarrow-17.0.0-cp311-cp311-win_amd64.whl.metadata (3.4 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp311-cp311-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl.metadata (14 kB)
  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
  Using cached soxr-0.5.0.post1-cp311-cp311-win_amd64.whl.metadata (5.6 kB)
  Using cached torch-2.5.0-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
Using cached accelerate-1.0.1-py3-none-any.whl (3

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.1 which is incompatible.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.28.2 which is incompatible.


In [6]:
import gradio as gr
# import function from another file in this repository, or copy function into this file. 

In [24]:
# placeholder function for testing here:

# import all dependencies and libraries
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from transformers import pipeline


# Load environment variables.
load_dotenv()
# Set the model name for our LLMs.
GEMINI_MODEL = "gemini-1.5-flash"
# Store the API key in a variable.
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")


# instatiate the LLM
llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.5)
# create conversation chain with memory
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)


In [25]:
# function for quick tests:

def placeholder_fn(query, history):
    # add user's message to conversation history
    history.append(("User", query))
    # get response from AI
    result = conversation.predict(input=query)
    # add AI resposne to conversation history
    history.append(("Coach", result))
    # format conversation history for display
    formatted_history = "\n".join([f"{sender}: {msg}" for sender, msg in history])

    return formatted_history, history


In [37]:
# add voice-to-text input

transcriber = pipeline("automatic-speech=recognition", model = "oepnai/whisper-large-v3")

def transcribe(audio):
    if audio is not None: 
        result = transcriber(audio)
        return result['text']
    else: 
        return ""

def process_input(audio, text, history):
    message = transcribe(audio) if audio is not None else text
    return placeholder_fn(message, history)

OSError: oepnai/whisper-large-v3 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [20]:
# gradio UI 

chat_app = gr.Interface(
    fn = process_input, 
    inputs = [
        gr.Audio(source="microphone", type="filepath", label="Speak your message (optional)"),
        gr.Textbox(lines = 2, label = "Or type your message here"),
        gr.State([])
    ],
    outputs = [
        gr.Textbox(lines = 10, label="Coach AI response:", show_copy_button=True),
        gr.State()
    ],
    title="Coach AI",
    description="Chat with an AI using your voice or by typing. The AI remembers your conversation history."
)

In [21]:
# launch app locally

chat_app.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='hi my name is liz. ', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Liz, it's nice to meet you! I'm an AI, so I don't have a name, but I'm happy to chat with you. What can I do for you today? 😊 \n", additional_kwargs={}, response_metadata={}), HumanMessage(content='can I ask a question?', additional_kwargs={}, response_metadata={}), AIMessage(content="Of course, Liz! I love answering questions. 😊  What's on your mind? \n", additional_kwargs={}, response_metadata={}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='You told me your name is Liz! 😊  Is there anything else 